In [1]:
import xgboost as xgb
import pandas as pd
from matplotlib import pyplot as plt
import warnings
import numpy as np
import os
from sklearn.metrics import mean_absolute_error
os.environ['NUMEXPR_MAX_THREADS'] = '16'
os.environ['NUMEXPR_NUM_THREADS'] = '10'

In [2]:
df_train = pd.read_csv('data/train.csv', header=0)
df_test = pd.read_csv('data/test.csv', header=0)
submission = pd.read_csv('data/sample_submission.csv', header=0)

In [3]:
df_train.shape, df_test.shape

((150150, 9), (13860, 8))

In [4]:
df_train.fillna(0, inplace=True)
df_test.fillna(0, inplace=True)

In [5]:
df_store_sku_max_base_price = df_train.groupby(['store_id', 'sku_id']).agg({'base_price': 'max'}).reset_index()
df_store_sku_max_base_price.columns = ['store_id', 'sku_id', 'max_base_price']
print(df_store_sku_max_base_price.head())

df_store_sku_max_total_price = df_train.groupby(['store_id', 'sku_id']).agg({'total_price': 'max'}).reset_index()
df_store_sku_max_total_price.columns = ['store_id', 'sku_id', 'max_total_price']
print(df_store_sku_max_total_price.head())

   store_id  sku_id  max_base_price
0      8023  216233        140.3625
1      8023  216418        119.7000
2      8023  216419        119.7000
3      8023  216425        141.0750
4      8023  217390        174.5625
   store_id  sku_id  max_total_price
0      8023  216233         140.3625
1      8023  216418         119.7000
2      8023  216419         119.7000
3      8023  216425         141.0750
4      8023  217390         174.5625


In [6]:
df_train = pd.merge(df_train, df_store_sku_max_total_price, on=['store_id', 'sku_id'])
df_train['total_price_by_max_total_price'] = df_train['total_price']/df_train['max_total_price']

df_test = pd.merge(df_test, df_store_sku_max_total_price, on=['store_id', 'sku_id'])
df_test['total_price_by_max_total_price'] = df_test['total_price']/df_test['max_total_price']

df_train.fillna(0, inplace=True)
df_test.fillna(0, inplace=True)

In [7]:
df_train.head()

,record_ID,week,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold,max_total_price,total_price_by_max_total_price
0,1,17/01/11,8091,216418,99.0375,111.8625,0,0,20,120.4125,0.822485
1,1641,24/01/11,8091,216418,99.0375,111.8625,0,0,34,120.4125,0.822485
2,3277,31/01/11,8091,216418,96.9000,96.9000,0,0,10,120.4125,0.804734
3,4913,07/02/11,8091,216418,98.3250,98.3250,0,0,17,120.4125,0.816568
4,6553,14/02/11,8091,216418,106.8750,106.8750,0,0,24,120.4125,0.887574


In [8]:
df_train.shape, df_test.shape

((150150, 11), (13860, 10))

In [9]:
df_train['units_sold'] = np.log(df_train['units_sold'])

In [10]:
df_train['week'].sample(6)

140727    01/05/12
111051    22/08/11
102787    18/09/12
122931    07/08/12
94936     26/09/11
125418    04/12/12
Name: week, dtype: object

In [11]:
df_train['week'] = pd.to_datetime(df_train['week'], format='%d/%m/%y')
df_test['week'] = pd.to_datetime(df_test['week'], format='%d/%m/%y')

In [12]:
df_train.groupby(['store_id', 'sku_id'])['week'].count().reset_index()['week'].sum()

150150

In [13]:
df_train.groupby(['store_id', 'sku_id'])['week'].count().reset_index()['week'].min(), \
df_train.groupby(['store_id', 'sku_id'])['week'].count().reset_index()['week'].max()

(130, 130)

In [14]:
df_test.groupby(['store_id', 'sku_id'])['week'].count().reset_index()['week'].min(), \
df_test.groupby(['store_id', 'sku_id'])['week'].count().reset_index()['week'].max()

(12, 12)

In [15]:
print(df_train['sku_id'].unique().shape[0], df_test['sku_id'].unique().shape[0])

28 28


In [16]:
df_week = pd.concat((df_train[['week']], df_test[['week']]))
df_week = df_week.drop_duplicates()
df_week = df_week.sort_values('week')
df_week['week_num'] = np.arange(0, df_week.shape[0])

In [17]:
df_week.head()

,week,week_num
0,2011-01-17,0
1,2011-01-24,1
2,2011-01-31,2
3,2011-02-07,3
4,2011-02-14,4


In [18]:
df_train = pd.merge(df_train, df_week, on=['week'])
df_test = pd.merge(df_test, df_week, on=['week'])

In [19]:
df_train['week_of_year'] = df_train['week'].dt.week
df_test['week_of_year'] = df_test['week'].dt.week

In [20]:
df_train.shape, df_test.shape

((150150, 13), (13860, 12))

In [21]:
df_train, df_eval = df_train[df_train.week_num <= 117], df_train[df_train.week_num > 117]

In [22]:
df_train.columns

Index(['record_ID', 'week', 'store_id', 'sku_id', 'total_price', 'base_price',
       'is_featured_sku', 'is_display_sku', 'units_sold', 'max_total_price',
       'total_price_by_max_total_price', 'week_num', 'week_of_year'],
      dtype='object')

In [23]:
feature_cols = ['store_id', 'sku_id', 'total_price', 'base_price','is_featured_sku', 'is_display_sku', 'max_total_price', 'total_price_by_max_total_price', 'week_num', 'week_of_year']
label_col = 'units_sold'

In [24]:
dtrain = xgb.DMatrix(df_train[feature_cols], df_train[label_col])
deval = xgb.DMatrix(df_eval[feature_cols], df_eval[label_col])
dtest = xgb.DMatrix(df_test[feature_cols])

In [25]:
param = {
            'seed': 0,
            'colsample_bytree': 0.7,
            'silent': 1,
            'subsample': 0.7,
            'learning_rate': 0.01,
            'objective': 'reg:linear',
            'max_depth': 12,
            'min_child_weight': 100,
            'booster': 'gbtree',
            'eval_metric':'rmse'}

# the rest of settings are the same
watchlist = [(dtrain, 'train'), (deval, 'eval')]
num_round = 30000

bst = xgb.train(param, dtrain, num_round, watchlist, early_stopping_rounds=100,verbose_eval=10)

[0]	train-rmse:3.14431	eval-rmse:3.13755
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 100 rounds.
[10]	train-rmse:2.85564	eval-rmse:2.85429
[20]	train-rmse:2.59488	eval-rmse:2.59903
[30]	train-rmse:2.35999	eval-rmse:2.36874
[40]	train-rmse:2.14847	eval-rmse:2.16160
[50]	train-rmse:1.95774	eval-rmse:1.97506
[60]	train-rmse:1.78628	eval-rmse:1.80789
[70]	train-rmse:1.63206	eval-rmse:1.65713
[80]	train-rmse:1.49363	eval-rmse:1.52229
[90]	train-rmse:1.36963	eval-rmse:1.40210
[100]	train-rmse:1.25905	eval-rmse:1.29550
[110]	train-rmse:1.15951	eval-rmse:1.19888
[120]	train-rmse:1.07038	eval-rmse:1.11137
[130]	train-rmse:0.99160	eval-rmse:1.03533
[140]	train-rmse:0.92167	eval-rmse:0.96808
[150]	train-rmse:0.85937	eval-rmse:0.90865
[160]	train-rmse:0.80393	eval-rmse:0.85434
[170]	train-rmse:0.75546	eval-rmse:0.80781
[180]	train-rmse:0.71322	eval-rmse:0.76747
[190]	train-rmse:0.67534	eval-rmse:0.73015
[200]	t

[1860]	train-rmse:0.34092	eval-rmse:0.42534
[1870]	train-rmse:0.34078	eval-rmse:0.42535
[1880]	train-rmse:0.34060	eval-rmse:0.42524
[1890]	train-rmse:0.34046	eval-rmse:0.42524
[1900]	train-rmse:0.34024	eval-rmse:0.42517
[1910]	train-rmse:0.34006	eval-rmse:0.42517
[1920]	train-rmse:0.33987	eval-rmse:0.42526
[1930]	train-rmse:0.33970	eval-rmse:0.42526
[1940]	train-rmse:0.33953	eval-rmse:0.42530
[1950]	train-rmse:0.33937	eval-rmse:0.42526
[1960]	train-rmse:0.33921	eval-rmse:0.42520
[1970]	train-rmse:0.33904	eval-rmse:0.42510
[1980]	train-rmse:0.33889	eval-rmse:0.42500
[1990]	train-rmse:0.33872	eval-rmse:0.42500
[2000]	train-rmse:0.33850	eval-rmse:0.42491
[2010]	train-rmse:0.33836	eval-rmse:0.42486
[2020]	train-rmse:0.33821	eval-rmse:0.42488
[2030]	train-rmse:0.33804	eval-rmse:0.42491
[2040]	train-rmse:0.33788	eval-rmse:0.42489
[2050]	train-rmse:0.33775	eval-rmse:0.42486
[2060]	train-rmse:0.33757	eval-rmse:0.42493
[2070]	train-rmse:0.33742	eval-rmse:0.42489
[2080]	train-rmse:0.33728	eval-r

In [26]:
bst.best_iteration

2236

In [27]:
dtrain = xgb.DMatrix(pd.concat((df_train[feature_cols], df_eval[feature_cols])), pd.concat((df_train[label_col], df_eval[label_col])))

In [28]:
# the rest of settings are the same
watchlist = [(dtrain, 'train')]
num_round = bst.best_iteration

# training and evaluation
bst = xgb.train(param, dtrain, num_round, watchlist,verbose_eval=10)
preds = bst.predict(dtest)

[0]	train-rmse:3.14363
[10]	train-rmse:2.85498
[20]	train-rmse:2.59452
[30]	train-rmse:2.35954
[40]	train-rmse:2.14772
[50]	train-rmse:1.95745
[60]	train-rmse:1.78576
[70]	train-rmse:1.63159
[80]	train-rmse:1.49280
[90]	train-rmse:1.36887
[100]	train-rmse:1.25799
[110]	train-rmse:1.15881
[120]	train-rmse:1.07005
[130]	train-rmse:0.99113
[140]	train-rmse:0.92049
[150]	train-rmse:0.85779
[160]	train-rmse:0.80282
[170]	train-rmse:0.75447
[180]	train-rmse:0.71168
[190]	train-rmse:0.67482
[200]	train-rmse:0.64167
[210]	train-rmse:0.61289
[220]	train-rmse:0.58746
[230]	train-rmse:0.56574
[240]	train-rmse:0.54697
[250]	train-rmse:0.53055
[260]	train-rmse:0.51671
[270]	train-rmse:0.50431
[280]	train-rmse:0.49352
[290]	train-rmse:0.48411
[300]	train-rmse:0.47604
[310]	train-rmse:0.46868
[320]	train-rmse:0.46239
[330]	train-rmse:0.45667
[340]	train-rmse:0.45132
[350]	train-rmse:0.44685
[360]	train-rmse:0.44264
[370]	train-rmse:0.43856
[380]	train-rmse:0.43501
[390]	train-rmse:0.43152
[400]	train

In [29]:
submission = pd.DataFrame({'record_ID':df_test['record_ID'], 'units_sold':preds})

In [30]:
# submission.loc[submission['units_sold'] < 0, 'units_sold'] = 1
submission['units_sold'] = np.exp(submission['units_sold'])

In [31]:
submission.shape

(13860, 2)

In [32]:
submission[['record_ID', 'units_sold']].to_csv('final.csv', index=None)